<a href="https://colab.research.google.com/github/emmacwatts/AutoTagsCRISPR/blob/main/FormattingTFsdf_and_BLASTcheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def make_dataframe_from_TFs_list(TF_list, ref_genome, annotation):
    '''
    Extracts information and sequence region for genes of interest (TFs) for design of primers per gene.

    Input: 
      TFs_list: excel file of query sequences with Gene_ID and Transcript_ID
      ref_genome: fasta file for reference genome
      annotation: .gtf file for ref_genome
    
    Output
      TFsdf: dataframe of TF information and sequences
      TFsdict_of_dict: TFsdf as a dictionary of dictionaries per index
    '''

    import pandas as pd
    from Bio import SeqIO

    #This is the input file containing the TFs we want to query
    #Imported as a pandas dataframe
    queryTFsdf = pd.read_excel(TF_list)

    #This is the .gtf file with annotations for each gene on the reference genome
    refAnnotationsHeaders = ["Chromosome", "Source", "Gene_Region", "Start", "Stop", "Score", "Strand", "Frame", "Attribute"]
    refGenomeAnnotation = pd.read_csv(annotation, sep = "\t", header = None, index_col = False, names = refAnnotationsHeaders)

    #This is the FASTA file of the reference genome sequence
    refSeqPerChromosome = {}
    for seq in SeqIO.parse(open(ref_genome), 'fasta'):
        refSeqPerChromosome[seq.id] = seq.seq
    
    #This is to reformat the "Attribute" category in refGenomeAnnotation, to extract Gene_ID, Gene_Symbol, and Transcript ID
    index = 0

    #Add new categories to the dataframe
    refGenomeAnnotation = refGenomeAnnotation.assign(Gene_ID = "", Gene_Symbol = "", Transcript_ID = "")

    #For each attribute value, extract the gene ID and symbol and add this to the new categories
    for attribute in refGenomeAnnotation['Attribute']:
        fullatt = (refGenomeAnnotation.loc[index]["Attribute"]).replace(";", "")
        fullatt = fullatt.replace('"', "")
        fullattsplit = fullatt.split(" ")
        refGenomeAnnotation.at[index,"Gene_ID"] = fullattsplit[1]
        refGenomeAnnotation.at[index,"Gene_Symbol"] = fullattsplit[3]
        if len(fullattsplit) == 8:
            refGenomeAnnotation.at[index,"Transcript_ID"] = fullattsplit[5]
        index+=1

    #Delete Attributes category
    del refGenomeAnnotation["Attribute"]

    #Select only rows that TFs are in, and keep only the start and stop codon gene regions

    refGenomeAnnotation = refGenomeAnnotation.loc[refGenomeAnnotation["Gene_Region"].isin(["start_codon", "stop_codon"])]

    TFsdf = refGenomeAnnotation[["Gene_ID", "Gene_Symbol", "Transcript_ID", "Chromosome", "Gene_Region", "Start", "Stop", "Strand"]].loc[refGenomeAnnotation["Gene_ID"].isin(queryTFsdf["Flybase_ID"])]

    #Add reference genome sequence per gene region
    #This will correspond to 1.3kb upstream and downstream of ATG/stop codon 
    TFsdf = TFsdf.assign(Reference_Seq = "")

    for index, rowcontents in TFsdf.iterrows():
        if rowcontents["Strand"] == "+":

            #Define 2.6kb gene region
            regionStart = rowcontents["Start"] - 1701
            regionStop = rowcontents["Stop"] + 1700

            #Add reference sequence
            TFsdf.at[index,"Reference_Seq"] = str(refSeqPerChromosome[rowcontents["Chromosome"]][regionStart:regionStop])

        if rowcontents["Strand"] == "-":

            #Define 2.6kb gene region
            regionStart = rowcontents["Start"] - 1701
            regionStop = rowcontents["Stop"] + 1700

            #Add reference sequence
            refPosStrandSeq = str(refSeqPerChromosome[rowcontents["Chromosome"]][regionStart:regionStop]) #This is the + strand seq, so goes from end to beginning
            TFsdf.at[index,"Reference_Seq"] = revComp(refPosStrandSeq)

    #re-index
    TFsdf = TFsdf.reset_index()
    del TFsdf["index"]

    #Create a dictionary of dictionaries for the df
    TFsdict_of_dict = {}
    for index, rowcontents in TFsdf.iterrows():
        TFsdict_of_dict[index] = rowcontents.to_dict()

    return TFsdf, TFsdict_of_dict

# BLAST Function

In [ ]:
#Primers output will be a dataframe, for BLAST we need FASTA:
def dftoFASTA(primersdf, transgenicFile):
  """
  Reformats a primers dataframe to the appropriate format for a BLAST search. 

  Input: 
    primersdf: a dataframe of primers
    transgenicFile: filename and path for fasta file to be created

  Output: transgenicFile - this is a fasta file of the primers identified by all other rows in the dataframe.

  """
  import pandas as pd
  from Bio import SeqIO

  primersFASTA = open(transgenicFile, "w")

  #Per row in the dataframe, concatenate all information as a header, and primer sequence as sequence
  #Write this into the fasta file as>header \n sequence
  for index, rowcontents in primersdf.iterrows():
    split = (str(rowcontents).split())
    header = (">" + "primer" + str(index))
    sequence = split[-7]

    primersFASTA.write(str(header) + "\n" + str(sequence) + "\n")

  primersFASTA.close()


In [ ]:
#use shell to run BLAST on primers against custom transgenic reference file
%%shell

#I've made a custom transgenic reference FASTA which uses the appropriate transgenic strain for each chromosome
#(has chromosome 3 from nos-Cas9_on_2 and all other chromosomes from nos-Cas9_on_3)
#Use this to BLAST against - "transgenicRef.fasta"

#install BLAST (if necessary)
gunzip 'ncbi-blast-2.13.0+-x64-linux.tar.gz'
tar -xvf 'ncbi-blast-2.13.0+-x64-linux.tar'
#change directory to ncbi-blast-2.13.0+/

#run BLAST
makeblastdb -in transgenicRef.fasta -dbtype nucl -parse_seqids 
blastn -query queryprimers.fasta -subject transgenicRef.fasta -evalue 1e-23 -outfmt 6 -out primerblast.txt -max_target_seqs 2
  #This output format is as simple as possible, so that we get as small a file when querying many primers at once

In [ ]:
def primerCountBLAST(BLASTresults, primersdf):
  '''
  Checks for the number of exact matches of a primer against a reference genome.

  Input:
  BLASTresults = txt file of BLAST results in -outfmt 6
  primersdf = p

  Output:
  faultyPrimers: the primers from primersdf that match zero times or more than once to the reference genome. Given in dataframe format.
  '''  
  import pandas as pd
  from Bio import SeqIO

  #Append the primers df with a new column counting the number of instances of that primer against the reference genome
  primersdf["Match Count"] = "0"

  #Open BLAST results
  primerBLAST = open(BLASTresults, "r")
  primerBLASTlines = primerBLAST.readlines()

  #Each 100% match will be labelled with the primer name, so count the instances of each 
  #primer name in the file and append this to "Match Count" in primersdf
  for line in primerBLASTlines:
    lineattributes = (line.split("\t"))
    primerindex = int(lineattributes[0].replace("primer", ""))
    currentcount = primersdf.at[primerindex, "Match Count"]
    primersdf.at[primerindex, "Match Count"] = int(currentcount) + 1
  
  #Keep only faulty primers
  faultyPrimers = primersdf
  for index, rowcontents in faultyPrimers.iterrows():
    if rowcontents["Match Count"] == 1:
      faultyPrimers.drop([index], axis = 0, inplace = True)

  return faultyPrimers